[Tensorflow official guide: Ragged Tensors](https://tensorflow.google.cn/guide/ragged_tensors)<br>
运行环境：tf-1.15.0-rc1

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import math
import tensorflow as tf
tf.enable_eager_execution()

In [2]:
digits = tf.ragged.constant([[3, 1, 4, 1], [], [5, 9, 2], [6], []])
words = tf.ragged.constant([["So", "long"], ["thanks", "for", "all", "the", "fish"]])
print(tf.add(digits, 3))  # broadcast, equivalent to digits + 3
print(tf.reduce_mean(digits, axis=1))
print(tf.concat([digits, [[5, 3]]], axis=1))
print(tf.tile(digits, [1, 2]))
print(tf.strings.substr(words, 0, 2))

<tf.RaggedTensor [[6, 4, 7, 4], [], [8, 12, 5], [9], []]>
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
tf.Tensor([2.25              nan 5.33333333 6.                nan], shape=(5,), dtype=float64)
<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9, 2], [6], [], [5, 3]]>
<tf.RaggedTensor [[3, 1, 4, 1, 3, 1, 4, 1], [], [5, 9, 2, 5, 9, 2], [6, 6], []]>
<tf.RaggedTensor [[b'So', b'lo'], [b'th', b'fo', b'al', b'th', b'fi']]>


In [3]:
# slice as normal tensors
print(digits[0])

tf.Tensor([3 1 4 1], shape=(4,), dtype=int32)


In [4]:
# elementwise transformation
times_two_plus_one = lambda x: x * 2 + 1
print(tf.ragged.map_flat_values(times_two_plus_one, digits))

<tf.RaggedTensor [[7, 3, 9, 3], [], [11, 19, 5], [13], []]>


In [5]:
# build raged tensor by split rowids
print(tf.RaggedTensor.from_value_rowids(
    values=[3, 1, 4, 1, 5, 9, 2, 6],
    value_rowids=[0, 0, 0, 0, 2, 2, 2, 3]))

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9, 2], [6]]>


In [6]:
# build ragged tensor by define row length
print(tf.RaggedTensor.from_row_lengths(
    values=[3, 1, 4, 1, 5, 9, 2, 6],
    row_lengths=[4, 0, 3, 1]))

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9, 2], [6]]>


In [7]:
# build ragged tensor by interval splits
print(tf.RaggedTensor.from_row_splits(
    values=[3, 1, 4, 1, 5, 9, 2, 6],
    row_splits=[0, 4, 4, 7, 8]))

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9, 2], [6]]>


In [8]:
# ragged tensor can have variant-length lists, but must all have 
# the same type and same nesting depth
try:
  tf.ragged.constant([["one", "two"], [3, 4]])              # bad: multiple types
except ValueError as exception:
  print(exception)

Can't convert Python sequence with mixed types to Tensor.


In [9]:
try:
  tf.ragged.constant(["A", ["B", "C"]])                     # bad: multiple nesting depths
except ValueError as exception:
  print(exception)

all scalar values must have the same nesting depth


### tensor type conversion

In [20]:
# ragged tensor to common tensor and sparse
ragged_sentences = tf.ragged.constant([
    ['Hi'], ['Welcome', 'to', 'the', 'fair'], ['Have', 'fun']])
print(ragged_sentences.to_tensor(default_value=''))
print(ragged_sentences.to_sparse())

tf.Tensor(
[[b'Hi' b'' b'' b'']
 [b'Welcome' b'to' b'the' b'fair']
 [b'Have' b'fun' b'' b'']], shape=(3, 4), dtype=string)
SparseTensor(indices=tf.Tensor(
[[0 0]
 [1 0]
 [1 1]
 [1 2]
 [1 3]
 [2 0]
 [2 1]], shape=(7, 2), dtype=int64), values=tf.Tensor([b'Hi' b'Welcome' b'to' b'the' b'fair' b'Have' b'fun'], shape=(7,), dtype=string), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


In [21]:
x = [[1, 3, -1, -1], [2, -1, -1, -1], [4, 5, 8, 9]]
print(tf.RaggedTensor.from_tensor(x, padding=-1))
st = tf.SparseTensor(indices=[[0, 0], [2, 0], [2, 1]],
                     values=['a', 'b', 'c'],
                     dense_shape=[3, 3])
print(tf.RaggedTensor.from_sparse(st))

<tf.RaggedTensor [[1, 3], [2], [4, 5, 8, 9]]>
<tf.RaggedTensor [[b'a'], [], [b'b', b'c']]>


In [24]:
rt = tf.ragged.constant([[1, 2], [3, 4, 5], [6], [], [7]])
print(rt.to_list())

[[1, 2], [3, 4, 5], [6], [], [7]]
